In [1]:

import warnings
warnings.filterwarnings('ignore')
import os
import seaborn as sns
import snowflake.connector
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
# import lightgbm and xgboost 
import lightgbm as lgb 
from lightgbm import LGBMRegressor
import xgboost as xgb 
from xgboost import plot_importance
from matplotlib import pyplot

from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score

#RMSE/MAE computation using sklearn library
from sklearn.metrics import mean_squared_error, mean_absolute_error
import scipy
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
import ipywidgets as widgets
from IPython.display import display, clear_output

In [2]:
# -- change time period to refresh data 
# create   table scratch.riders.historical_Utilsation_AOD_daily_small_city as
# with base as
# (
# select 
# distinct 
# lower(city_name) as city_name
# ,case when zone_code in
# ('ABAR','AKW','AMR','AMZ','ANH','AQ','AQPP','AQS','ATW','AWH','DBB','DCH','DD11','DDO','DEIR','DFC','DIC','DMEY','DSO',
# 'DSW','DT','DWS','FCOW','HCC','HPDT','JMH','KAR','MRDF','NAS','RAK','UMS','WAA','ZBL') then  'Deira and Downtown Cluster'
# when zone_code in (
# 'AR','DBRS','DDH','DEGC','DHS','DHS2','DICP','DIP','DJP','DM','DSB','DTS','HPJLT','HPMC','IBN','JBLI','JGE','JI',
# 'JLT','JVC','JVT','MDW8','PC','PR','RMRM','SC','SPR','SS','TM','TWM') then 'Sportcity and Marina Cluster'
# else city_name end as Cluster_Assigned
# ,Date(START_OF_PERIOD_LOCAL) as dt
# -- ,CAST(EXTRACT(HOUR FROM TO_TIMESTAMP(START_OF_PERIOD_LOCAL )) AS INT)as order_created_hour
# ,case 
#      when CAST(EXTRACT(HOUR FROM START_OF_PERIOD_LOCAL) AS INT)  in (12, 13) and DAYNAME(START_OF_PERIOD_LOCAL) in ('Mon', 'Tue', 'Wed','Thu','Fri') then 'Peak_lunch'
#      when CAST(EXTRACT(HOUR FROM START_OF_PERIOD_LOCAL) AS INT)  in (19, 20) and DAYNAME(START_OF_PERIOD_LOCAL) in ('Mon', 'Tue', 'Wed','Thu') then 'Peak Dinner'
#      when CAST(EXTRACT(HOUR FROM START_OF_PERIOD_LOCAL) AS INT)  in (19, 20) and DAYNAME(START_OF_PERIOD_LOCAL) in ('Fri','Sat','Sun') then 'Super Peak Dinner'
#      else 'Other' end as shift

# ,coalesce(sum(ORDERS_DELIVERED),0)as orders_delievered

# ,COALESCE(SUM(RET_MINS_SUM ), 0) / NULLIF(COALESCE(SUM(ret_mins_cnt ), 0), 0) AS RET_AVG
# ,COALESCE(SUM(aod_mins_sum ), 0) / NULLIF(COALESCE(SUM(aod_mins_cnt ), 0), 0) AS AOD_AVG
# ,NULLIFZERO(sum(ORDERS_ASAP))/NULLIFZERO(sum(RIDERS_AVAILABLE_IN_HOUR)) as TP_1

# ,NULLIFZERO(sum(RIDER_HOURS_ON_ORDERS_DHW_SUM))/ NULLIFZERO(sum(RIDER_HOURS_WORKED)) as Utilisation_rate_1
# ,coalesce(sum(TOTAL_ORDER_TIME_PHYSICAL_ZONE_HOURS),0)/NULLIF(COALESCE(sum(TOTAL_TIME_PHYSICAL_ZONE_HOURS ),0),0) as UR_zone_corrected
# from
# PRODUCTION.AGGREGATE.AGG_ZONE_DELIVERY_METRICS_HOURLY
# where  date (start_of_period_local) between '2024-05-01' and '2024-06-30'
# -- date (start_of_period_local) between 
# --     dateadd('month', -6, date_trunc('month',  current_date()))
# --     and  LAST_DAY(dateadd('month', -1, date_trunc('month',  current_date())), MONTH)
# and CNT_ERAT >0 -- use this filter to get only active zone
# and case when UPPER(country_name) ='KUWAIT' and date (start_of_period_local) between '2024-03-10' and '2024-04-10' then true else false end  =false
# group by 1,2,3,4
# )
# ,TP_calc as
# (
# select 
# *
# ,coalesce(UR_zone_corrected*(60/ RET_avg),TP_1)  as TP_zone_corrected
# from 
# base 
# )
# select *exclude(TP_1) from TP_calc;


# -- GRANT SELECT ON TABLE scratch.riders.historical_Utilsation_AOD_daily_small_city TO ROLE BI_DEVELOPMENT;





In [3]:
pd_query="""
select * from scratch.riders.historical_Utilsation_AOD_daily_small_city 
"""

conn = snowflake.connector.connect(
        user='Richa.kumari@deliveroo.com',
        password=os.getenv('password'),
        account="deliveroo.eu-central-1",
        authenticator='externalbrowser',
        warehouse="BI_DEVELOPMENT",
        role="BI_DEVELOPMENT",
    )


cur = conn.cursor()
cur.execute(pd_query)
df = cur.fetch_pandas_all()


Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://accounts.google.com/o/saml2/idp?idpid=C01jnk96c&SAMLRequest=nZLRbtowFIZfJfKuEzuBrcUiIAqtiEQHg4DU3nmOoV4cn8x2CN3Tz4QidRftxS4iRc53jr%2BT%2FwzHp0oFR2GsBJ2iOCIoEJpDIfUhRdv8IbxFgXVMF0yBFil6FRaNR0PLKlXTSeNe9Fr8boR1gW%2BkLe0%2BpKgxmgKz0lLNKmGp43QzeVzQJCK0NuCAg0LvSj6vYNYK47zhtaSw0uu9OFdTjNu2jdpeBOaAE0IIJgPsqTPy5cqf%2FEwf8DEm%2FTPvCY%2Bv3tzupL78gs%2B0fl4gS%2Bd5vgpXy02OgslVdQraNpUwG2GOkovtenERsN5gdr%2FIdvfr5TISTciFdoapMI6shnavWCk4VHXjfOvIv%2BG9KLCCg%2FTTZ7MU1aUsdrcnMOVTki3ueqDmhS7z3k0M%2FHl%2B7Kl9tt0c149Pf2439Y%2BSo2B3jTc5x5tZ24hMn0N1%2Fogk%2FZDchEmSxz1KBvQriQZJ%2FxkFMx%2Bq1Mx1lVdzxjk02tnoAHBQovMDfE4wwbKox%2F6RRTol8S9dDr5xdNkU2t1pRv81%2FxC%2Fb%2FG2ed99GNlsBUry1%2BABTMXcx1nFUdydyCLcdygVFZNqUhRGWOszUwraqRHM%2BQV3phEIjy63%2Frvio78%3D&RelayState=5

In [4]:
print(df.shape)
df.head(2)

df['id']=df['CLUSTER_ASSIGNED']+df['SHIFT'].astype(str)+df['DT'].astype(str)
df['DT']=pd.to_datetime(df['DT'], format='%d%b%Y:%H:%M:%S.%f')
df.head(2)
base_df=df.copy()

(1318, 10)


In [5]:
#  drop the rows with null AOD and UR
base_df=base_df[base_df['UR_ZONE_CORRECTED'].notna()]
print(base_df.shape)
base_df=base_df[base_df['AOD_AVG'].notna()]
print(base_df.shape)
null_df= ((base_df.isna().sum()/ len(base_df))*100).sort_values(ascending=False)
null_df

(1318, 11)
(1318, 11)


CITY_NAME             0.0
CLUSTER_ASSIGNED      0.0
DT                    0.0
SHIFT                 0.0
ORDERS_DELIEVERED     0.0
RET_AVG               0.0
AOD_AVG               0.0
UTILISATION_RATE_1    0.0
UR_ZONE_CORRECTED     0.0
TP_ZONE_CORRECTED     0.0
id                    0.0
dtype: float64

In [6]:
base_df.head(1)

,CITY_NAME,CLUSTER_ASSIGNED,DT,SHIFT,ORDERS_DELIEVERED,RET_AVG,AOD_AVG,UTILISATION_RATE_1,UR_ZONE_CORRECTED,TP_ZONE_CORRECTED,id
0,ajman,Ajman,2024-06-24,Other,190,25.185088,31.292754,0.532818,0.563128,1.341576,AjmanOther2024-06-24


In [7]:

base_df.CLUSTER_ASSIGNED.unique()

array(['Ajman', 'Abu Dhabi', 'Dubai', 'Al Ain', 'Ras Al Khaimah',
       'Sharjah', 'Doha', 'Kuwait'], dtype=object)

In [7]:
#  Filter for only smaller cities 
small_cities=['Al Ain','Ras Al Khaimah', 'Ajman']

filtered_df=base_df[base_df['CLUSTER_ASSIGNED'].isin(small_cities)]
filtered_df.CLUSTER_ASSIGNED.unique()

array(['Ajman', 'Al Ain', 'Ras Al Khaimah'], dtype=object)

# # Linear Regression Results for UR prediction


In [8]:
df = filtered_df.copy()
def cap_values(series):
    lower_cap = series.quantile(0.01)
    upper_cap = series.quantile(0.9999)
    return np.clip(series, lower_cap, upper_cap)

def calculate_prediction_percentage_errors(y_true, y_pred):
    overpredictions = np.sum(y_pred >= y_true)
    underpredictions = np.sum(y_pred < y_true)
    total_predictions = len(y_true)
    overprediction_pct = (overpredictions / total_predictions) * 100
    underprediction_pct = (underpredictions / total_predictions) * 100
    return overprediction_pct, underprediction_pct


def build_and_evaluate_model(df):
    clusters = df['CLUSTER_ASSIGNED'].unique()
    
    results = []
    df['Predictions_linear_regression'] = np.nan  # Add a column for predictions
    
    for cluster in clusters:
        cluster_df = df[df['CLUSTER_ASSIGNED'] == cluster].copy()
        
        # Cap the values
        cluster_df['AOD_AVG'] = cap_values(cluster_df['AOD_AVG'])
        cluster_df['UR_ZONE_CORRECTED'] = cap_values(cluster_df['UR_ZONE_CORRECTED'])
        
        # Prepare data for linear regression
        X = cluster_df[['AOD_AVG']]
        y = cluster_df['UR_ZONE_CORRECTED']
        
        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
        
        # Train the model
        model = LinearRegression()
        model.fit(X, y)
        
        # Make predictions
        predictions = model.predict(X)
        df.loc[df['CLUSTER_ASSIGNED'] == cluster, 'Predictions_linear_regression'] = predictions
        
        # Calculate evaluation metrics
        mae = mean_absolute_error(y, predictions)
        mape = mean_absolute_percentage_error(y, predictions)
        # print((eval_throughput(y,predictions)))
        r2 = r2_score(y, predictions)
        rmse = np.sqrt(mean_squared_error(y, predictions))
        overprediction_pct, underprediction_pct = calculate_prediction_percentage_errors(y, predictions)
        
        

        # Get the regression equation
        intercept = model.intercept_
        coefficient = model.coef_[0]
        equation = f"UR_ZONE_CORRECTED = {intercept:.4f} + {coefficient:.4f} * AOD_AVG"
        
        results.append({
            'CLUSTER_ASSIGNED': cluster,
            'Equation': equation,
            'MAE': mae,
            'MAPE': mape,
            'R2': r2,
            'RMSE': rmse,
            'Overprediction %': overprediction_pct,
            'Underprediction %': underprediction_pct
            
        })
    
    return pd.DataFrame(results), df

linear_results_df_small_UAE_cities, linear_df_with_predictions = build_and_evaluate_model(df)

linear_results_df_small_UAE_cities



,CLUSTER_ASSIGNED,Equation,MAE,MAPE,R2,RMSE,Overprediction %,Underprediction %
0,Ajman,UR_ZONE_CORRECTED = 0.0175 + 0.0163 * AOD_AVG,0.062107,0.119378,0.189063,0.081023,50.909091,49.090909
1,Al Ain,UR_ZONE_CORRECTED = -0.2505 + 0.0192 * AOD_AVG,0.075910,0.196092,0.398669,0.096802,43.292683,56.707317
2,Ras Al Khaimah,UR_ZONE_CORRECTED = -0.1954 + 0.0235 * AOD_AVG,0.052770,0.096049,0.536762,0.069883,43.902439,56.097561


In [9]:
linear_df_with_predictions.groupby(['CLUSTER_ASSIGNED','SHIFT'])[['UR_ZONE_CORRECTED','Predictions_linear_regression']].mean()
 

UR_ZONE_CORRECTED  \
CLUSTER_ASSIGNED SHIFT                                  
Ajman            Other                       0.509209   
                 Peak Dinner                 0.587560   
                 Peak_lunch                  0.562154   
                 Super Peak Dinner           0.545034   
Al Ain           Other                       0.504598   
                 Peak Dinner                 0.534221   
                 Peak_lunch                  0.333602   
                 Super Peak Dinner           0.525419   
Ras Al Khaimah   Other                       0.572095   
                 Peak Dinner                 0.577033   
                 Peak_lunch                  0.588775   
                 Super Peak Dinner           0.549750   

                                    Predictions_linear_regression  
CLUSTER_ASSIGNED SHIFT                                             
Ajman            Other                                   0.531670  
                 Peak Dinner                             0.554206  
                 Peak_lunch                              0.553207  
                 Super Peak Dinner                       0.554535  
Al Ain           Other                                   0.470740  
                 Peak Dinner                             0.463826  
                 Peak_lunch                              0.447386  
                 Super Peak Dinner                       0.515122  
Ras Al Khaimah   Other                                   0.557587  
                 Peak Dinner                             0.563347  
                 Peak_lunch                              0.608473  
                 Super Peak Dinner                       0.569901

##   Interactive output for the above method

In [10]:

def display_equation_and_predict(cluster):
    clear_output()  # Clear previous output
    
    # Display the regression equation for the selected cluster
    result_row = linear_results_df_small_UAE_cities[linear_results_df_small_UAE_cities['CLUSTER_ASSIGNED'] == cluster].iloc[0]
    equation = result_row['Equation']
    print(f"Equation for Cluster {cluster}: {equation}")
    
    # Create an input box for AOD value
    aod_value = widgets.FloatText(
        value=0,
        description='AOD Value:',
        disabled=False
    )
    
    # Create an output area for displaying the prediction
    output = widgets.Output()
    
    # Function to handle button click and display prediction
    def on_button_clicked(b):
        with output:
            clear_output()  # Clear previous output in the output area
            aod = aod_value.value
            intercept, coefficient = result_row['Equation'].split(' = ')[1].split(' + ')
            coefficient = float(coefficient.split(' * ')[0])
            intercept = float(intercept)
            prediction = intercept + coefficient * aod
            print(f"Predicted UR_ZONE_CORRECTED for AOD {aod}: {prediction:.2f}")
    
    # Create a button for making predictions
    button = widgets.Button(
        description='Predict UR_ZONE_CORRECTED',
        disabled=False,
        button_style='',  # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Click to make prediction',
        icon='check'
    )
    
    button.on_click(on_button_clicked)
    
    # Display the input box, button, and output area
    display(aod_value, button, output)

# Create a dropdown for selecting cluster
cluster_dropdown = widgets.Dropdown(
    options=linear_results_df_small_UAE_cities['CLUSTER_ASSIGNED'].unique(),
    value=linear_results_df_small_UAE_cities['CLUSTER_ASSIGNED'].unique()[0],
    description='Cluster:',
    disabled=False
)

# Display interactive widgets
widgets.interactive(display_equation_and_predict, cluster=cluster_dropdown)




interactive(children=(Dropdown(description='Cluster:', options=('Ajman', 'Al Ain', 'Ras Al Khaimah'), value='A…

In [11]:
linear_results_df_small_UAE_cities.to_csv('eqn_small_cities.csv')

#  Predict for all other cities

In [12]:

# create   table scratch.riders.historical_Utilsation_AOD_daily as
# with base as
# (
# select 
# distinct 
# lower(city_name) as city_name
# ,case when zone_code in
# ('ABAR','AKW','AMR','AMZ','ANH','AQ','AQPP','AQS','ATW','AWH','DBB','DCH','DD11','DDO','DEIR','DFC','DIC','DMEY','DSO',
# 'DSW','DT','DWS','FCOW','HCC','HPDT','JMH','KAR','MRDF','NAS','RAK','UMS','WAA','ZBL') then  'Deira and Downtown Cluster'
# when zone_code in (
# 'AR','DBRS','DDH','DEGC','DHS','DHS2','DICP','DIP','DJP','DM','DSB','DTS','HPJLT','HPMC','IBN','JBLI','JGE','JI',
# 'JLT','JVC','JVT','MDW8','PC','PR','RMRM','SC','SPR','SS','TM','TWM') then 'Sportcity and Marina Cluster'
# else city_name end as Cluster_Assigned
# ,Date(START_OF_PERIOD_LOCAL) as dt
# -- ,CAST(EXTRACT(HOUR FROM TO_TIMESTAMP(START_OF_PERIOD_LOCAL )) AS INT)as order_created_hour
# ,case 
#      when CAST(EXTRACT(HOUR FROM START_OF_PERIOD_LOCAL) AS INT)  in (12, 13) and DAYNAME(START_OF_PERIOD_LOCAL) in ('Mon', 'Tue', 'Wed','Thu','Fri') then 'Peak_lunch'
#      when CAST(EXTRACT(HOUR FROM START_OF_PERIOD_LOCAL) AS INT)  in (19, 20) and DAYNAME(START_OF_PERIOD_LOCAL) in ('Mon', 'Tue', 'Wed','Thu') then 'Peak Dinner'
#      when CAST(EXTRACT(HOUR FROM START_OF_PERIOD_LOCAL) AS INT)  in (19, 20) and DAYNAME(START_OF_PERIOD_LOCAL) in ('Fri','Sat','Sun') then 'Super Peak Dinner'
#      else 'Other' end as shift

# ,coalesce(sum(ORDERS_DELIVERED),0)as orders_delievered

# ,COALESCE(SUM(RET_MINS_SUM ), 0) / NULLIF(COALESCE(SUM(ret_mins_cnt ), 0), 0) AS RET_AVG
# ,COALESCE(SUM(aod_mins_sum ), 0) / NULLIF(COALESCE(SUM(aod_mins_cnt ), 0), 0) AS AOD_AVG
# ,NULLIFZERO(sum(ORDERS_ASAP))/NULLIFZERO(sum(RIDERS_AVAILABLE_IN_HOUR)) as TP_1

# ,NULLIFZERO(sum(RIDER_HOURS_ON_ORDERS_DHW_SUM))/ NULLIFZERO(sum(RIDER_HOURS_WORKED)) as Utilisation_rate_1
# ,coalesce(sum(TOTAL_ORDER_TIME_PHYSICAL_ZONE_HOURS),0)/NULLIF(COALESCE(sum(TOTAL_TIME_PHYSICAL_ZONE_HOURS ),0),0) as UR_zone_corrected
# from
# PRODUCTION.AGGREGATE.AGG_ZONE_DELIVERY_METRICS_HOURLY
# where  date (start_of_period_local) between '2023-09-01' and '2024-05-30'
# -- date (start_of_period_local) between 
# --     dateadd('month', -6, date_trunc('month',  current_date()))
# --     and  LAST_DAY(dateadd('month', -1, date_trunc('month',  current_date())), MONTH)
# and CNT_ERAT >0 -- use this filter to get only active zone
# and case when UPPER(country_name) ='KUWAIT' and date (start_of_period_local) between '2024-03-10' and '2024-04-10' then true else false end  =false
# group by 1,2,3,4
# )
# ,TP_calc as
# (
# select 
# *
# ,coalesce(UR_zone_corrected*(60/ RET_avg),TP_1)  as TP_zone_corrected
# from 
# base 
# )
# select *exclude(TP_1) from TP_calc;

# select count(*), count(distinct CLUSTER_ASSIGNED, dt,shift)  from  scratch.riders.historical_Utilsation_AOD_daily-- 6556 
# GRANT SELECT ON TABLE scratch.riders.historical_Utilsation_AOD_daily TO ROLE BI_DEVELOPMENT;





In [13]:
pd_query="""
select * from scratch.riders.historical_Utilsation_AOD_daily 
"""

conn = snowflake.connector.connect(
        user='Richa.kumari@deliveroo.com',
        password=os.getenv('password'),
        account="deliveroo.eu-central-1",
        authenticator='externalbrowser',
        warehouse="BI_DEVELOPMENT",
        role="BI_DEVELOPMENT",
    )


cur = conn.cursor()
cur.execute(pd_query)
base_df_big_cities = cur.fetch_pandas_all()


Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://accounts.google.com/o/saml2/idp?idpid=C01jnk96c&SAMLRequest=nZJNb%2BIwEIb%2FSuQ9J3ZSui0WAbHQaiNBSSGwEjeTGOrFsVOPQ0p%2FfR0%2BpO6hPezBkmW%2FM8%2FMvNMbvJXSO3ADQqsYhQFBHle5LoTaxWiZPfr3yAPLVMGkVjxGRw5o0O8BK2VFh7V9UXP%2BWnOwnkukgLYfMaqNopqBAKpYyYHanC6G0wmNAkIZADfW4dAlpADhWC%2FWVhTjpmmC5ibQZocjQggmXexUreQH%2BoSovmdURluda3kNeXM9fYEIMem0CKdwhPQS%2BEuo8wi%2Bo2zOIqC%2Fsyz109kiQ97w2t1IK6hLbhbcHETOl%2FPJuQBwFYwfJsnqYT6bBbz2c66sYdIPA1C62Uq257kuq9q61IG74S0vsNQ74QaWjGNU7UWxfu5Mpk%2FJKl3eQb0mk4OQr9kymYvNhr2PFtM%2Fh%2BnxfXYfPd92cuStrvZGrb0JQM0T1Zpq3ROJOj6586MoC29oSGjYDTrR7Rp5Y2eqUMyeIq%2BVszzXtbIQ7LTeSX6qT%2BPWkQiLohq4I4p4RMK%2Fat%2F9maPzptAT0%2FT%2Fq%2F8e%2FpzisnlPzoxknGop8qP3qE3J7NdehUF4ehGFvz1JKS%2BZkMOiMBzAeSalbkaGM%2BsW3JqaI9w%2FU%2F9d8f4H&RelayS

In [14]:
print(base_df_big_cities.shape)

base_df_big_cities['id']=base_df_big_cities['CLUSTER_ASSIGNED']+base_df_big_cities['SHIFT'].astype(str)+base_df_big_cities['DT'].astype(str)
base_df_big_cities['DT']=pd.to_datetime(base_df_big_cities['DT'], format='%d%b%Y:%H:%M:%S.%f')
base_df_big_cities.head(2)
base_df_big_cities=base_df_big_cities.copy()

(6556, 10)


In [15]:
#  drop the rows with null AOD and UR
base_df_big_cities=base_df_big_cities[base_df_big_cities['UR_ZONE_CORRECTED'].notna()]
print(base_df_big_cities.shape)
base_df_big_cities=base_df_big_cities[base_df_big_cities['AOD_AVG'].notna()]
print(base_df_big_cities.shape)


(6555, 11)
(6552, 11)


In [16]:
small_cities=['Al Ain','Ras Al Khaimah', 'Ajman']

big_cities_df=base_df_big_cities[~base_df_big_cities['CLUSTER_ASSIGNED'].isin(small_cities)]
big_cities_df.CLUSTER_ASSIGNED.unique()

array(['Kuwait', 'Doha', 'Deira and Downtown Cluster',
       'Sportcity and Marina Cluster', 'Abu Dhabi', 'Sharjah'],
      dtype=object)

In [17]:
big_city_df = big_cities_df.copy()

def cap_values(series):
    lower_cap = series.quantile(0.05)
    upper_cap = series.quantile(0.95)
    return np.clip(series, lower_cap, upper_cap)

def calculate_prediction_percentage_errors(y_true, y_pred):
    overpredictions = np.sum(y_pred >= y_true)
    underpredictions = np.sum(y_pred < y_true)
    total_predictions = len(y_true)
    overprediction_pct = (overpredictions / total_predictions) * 100
    underprediction_pct = (underpredictions / total_predictions) * 100
    return overprediction_pct, underprediction_pct


def build_and_evaluate_model(df):
    clusters = df['CLUSTER_ASSIGNED'].unique()
    
    results = []
    df['Predictions_linear_regression'] = np.nan  # Add a column for predictions
    
    for cluster in clusters:
        cluster_df = df[df['CLUSTER_ASSIGNED'] == cluster].copy()
        
        # Cap the values
        cluster_df['AOD_AVG'] = cap_values(cluster_df['AOD_AVG'])
        cluster_df['UR_ZONE_CORRECTED'] = cap_values(cluster_df['UR_ZONE_CORRECTED'])
        
        # Prepare data for linear regression
        X = cluster_df[['AOD_AVG']]
        y = cluster_df['UR_ZONE_CORRECTED']
        
        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
        
        # Train the model
        model = LinearRegression()
        model.fit(X, y)
        
        # Make predictions
        predictions = model.predict(X)
        df.loc[df['CLUSTER_ASSIGNED'] == cluster, 'Predictions_linear_regression'] = predictions
        
        # Calculate evaluation metrics
        mae = mean_absolute_error(y, predictions)
        mape = mean_absolute_percentage_error(y, predictions)
        # print((eval_throughput(y,predictions)))
        r2 = r2_score(y, predictions)
        rmse = np.sqrt(mean_squared_error(y, predictions))
        overprediction_pct, underprediction_pct = calculate_prediction_percentage_errors(y, predictions)
        
        

        # Get the regression equation
        intercept = model.intercept_
        coefficient = model.coef_[0]
        equation = f"UR_ZONE_CORRECTED = {intercept:.4f} + {coefficient:.4f} * AOD_AVG"
        
        results.append({
            'CLUSTER_ASSIGNED': cluster,
            'Equation': equation,
            'MAE': mae,
            'MAPE': mape,
            'R2': r2,
            'RMSE': rmse,
            'Overprediction %': overprediction_pct,
            'Underprediction %': underprediction_pct
            
        })
    
    return pd.DataFrame(results), df

linear_results_df_big_city, linear_df_with_predictions_big_city = build_and_evaluate_model(big_city_df)
linear_results_df_big_city

,CLUSTER_ASSIGNED,Equation,MAE,MAPE,R2,RMSE,Overprediction %,Underprediction %
0,Kuwait,UR_ZONE_CORRECTED = -0.5527 + 0.0323 * AOD_AVG,0.040501,0.069032,0.763835,0.052642,43.865031,56.134969
1,Doha,UR_ZONE_CORRECTED = -0.6321 + 0.0299 * AOD_AVG,0.070243,0.124039,0.472712,0.087701,48.849797,51.150203
2,Deira and Downtown Cluster,UR_ZONE_CORRECTED = -0.6930 + 0.0354 * AOD_AVG,0.035282,0.055849,0.660259,0.044328,46.278755,53.721245
3,Sportcity and Marina Cluster,UR_ZONE_CORRECTED = -0.7073 + 0.0379 * AOD_AVG,0.041531,0.063170,0.770216,0.052446,51.285521,48.714479
4,Abu Dhabi,UR_ZONE_CORRECTED = -0.8693 + 0.0407 * AOD_AVG,0.050433,0.080118,0.561048,0.063428,47.225981,52.774019
5,Sharjah,UR_ZONE_CORRECTED = -0.4706 + 0.0310 * AOD_AVG,0.057307,0.109250,0.430958,0.070065,52.108844,47.891156


In [18]:
#  collate the results of the 2
linear_df_with_predictions_big_city.groupby(['CLUSTER_ASSIGNED','SHIFT'])[['UR_ZONE_CORRECTED','Predictions_linear_regression']].mean()


UR_ZONE_CORRECTED  \
CLUSTER_ASSIGNED             SHIFT                                  
Abu Dhabi                    Other                       0.592476   
                             Peak Dinner                 0.738467   
                             Peak_lunch                  0.605541   
                             Super Peak Dinner           0.718588   
Deira and Downtown Cluster   Other                       0.604452   
                             Peak Dinner                 0.672245   
                             Peak_lunch                  0.679625   
                             Super Peak Dinner           0.686288   
Doha                         Other                       0.580465   
                             Peak Dinner                 0.649758   
                             Peak_lunch                  0.604271   
                             Super Peak Dinner           0.636379   
Kuwait                       Other                       0.615679   
                             Peak Dinner                 0.727153   
                             Peak_lunch                  0.522508   
                             Super Peak Dinner           0.727392   
Sharjah                      Other                       0.518400   
                             Peak Dinner                 0.554257   
                             Peak_lunch                  0.523999   
                             Super Peak Dinner           0.578296   
Sportcity and Marina Cluster Other                       0.580899   
                             Peak Dinner                 0.755459   
                             Peak_lunch                  0.638679   
                             Super Peak Dinner           0.777696   

                                                Predictions_linear_regression  
CLUSTER_ASSIGNED             SHIFT                                             
Abu Dhabi                    Other                                   0.608413  
                             Peak Dinner                             0.685964  
                             Peak_lunch                              0.639647  
                             Super Peak Dinner                       0.698797  
Deira and Downtown Cluster   Other                                   0.609210  
                             Peak Dinner                             0.667056  
                             Peak_lunch                              0.678639  
                             Super Peak Dinner                       0.678959  
Doha                         Other                                   0.592622  
                             Peak Dinner                             0.627185  
                             Peak_lunch                              0.613255  
                             Super Peak Dinner                       0.624044  
Kuwait                       Other                                   0.599389  
                             Peak Dinner                             0.706973  
                             Peak_lunch                              0.568711  
                             Super Peak Dinner                       0.711904  
Sharjah                      Other                                   0.517725  
                             Peak Dinner                             0.531786  
                             Peak_lunch                              0.534735  
                             Super Peak Dinner                       0.580442  
Sportcity and Marina Cluster Other                                   0.608997  
                             Peak Dinner                             0.708619  
                             Peak_lunch                              0.639599  
                             Super Peak Dinner                       0.774585

In [19]:
#  union the 2 equation output
linear_results_df_big_city
linear_results_df_small_UAE_cities
# Union the two DataFrames
union_df = pd.concat([linear_results_df_big_city, linear_results_df_small_UAE_cities], ignore_index=True)
union_df
union_df.to_csv('eqn_all_cities_july_vf.csv')
